In [49]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

# 读取训练数据

In [2]:
if __name__ == "__main__":
    # pandas 读入数据
    dataOnline = pd.read_csv('test1.csv')
    print(dataOnline.shape)

(299, 7)


# 数据预处理
## 设置输出值
## 选择Distance作为特征
## 划分测试机与训练集

In [3]:
dataOnline['result'] = dataOnline[['Coupon_id', 'Date']].apply(lambda x: 1 if pd.notnull(x[0]) and pd.notnull(x[1]) else 0, axis=1)

In [ ]:
# print(dataOnline)

In [4]:
dataOnline.Distance = dataOnline.Distance.fillna(9999)
dataOnline = dataOnline[dataOnline.Distance != 9999]
X = dataOnline[['Distance']]

In [5]:
y = dataOnline.result
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:
# print(x_train)
# print(y_train)

# 训练模型

In [6]:
classify = LogisticRegression()
classify.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

# 读入测试数据

In [51]:
dataTest = pd.read_csv('ccf_offline_stage1_test_revised.csv')

In [52]:
print(type(dataTest))

<class 'pandas.core.frame.DataFrame'>


In [53]:
print(dataTest.shape)

(113640, 6)


In [54]:
print(dataTest)

        User_id  Merchant_id  Coupon_id Discount_rate  Distance  Date_received
0       4129537          450       9983          30:5       1.0       20160712
1       6949378         1300       3429          30:5       NaN       20160706
2       2166529         7113       6928        200:20       5.0       20160727
3       2166529         7113       1808        100:10       5.0       20160727
4       6172162         7605       6500          30:1       2.0       20160708
5       4005121          450       9983          30:5       0.0       20160706
6       4347394          450       9983          30:5       0.0       20160716
7       3094273          760      13602          30:5       1.0       20160727
8       5139970          450       9983          30:5      10.0       20160729
9       3237121          760      13602          30:5       1.0       20160703
10      6224386          450       9983          30:5       3.0       20160716
11      6488578          760      13602          30:

# 测试数据预处理
## 提取测试数据Distance\

In [55]:
X = dataTest[['Distance']]
X= X.fillna(10)
print(type(X))
print(X.shape)

<class 'pandas.core.frame.DataFrame'>
(113640, 1)


# 计算 预测的概率

In [56]:
predict_proba_y = classify.predict_proba(X)

In [41]:
print(predict_proba_y)

[[0.60394453 0.39605547]
 [0.8780775  0.1219225 ]
 [0.75248187 0.24751813]
 ...
 [0.8780775  0.1219225 ]
 [0.56203788 0.43796212]
 [0.75248187 0.24751813]]


## 合并数据

In [42]:
dataTest = dataTest[['User_id', 'Coupon_id', 'Date_received']]
print(type(dataTest))
print(len(dataTest.columns))

<class 'pandas.core.frame.DataFrame'>
3


In [43]:
dataTest.insert(len(dataTest.columns), 'probability', predict_proba_y[:,1], True)

In [46]:
print(dataTest)

        User_id  Coupon_id  Date_received  probability
0       4129537       9983       20160712     0.396055
1       6949378       3429       20160706     0.121922
2       2166529       6928       20160727     0.247518
3       2166529       1808       20160727     0.247518
4       6172162       6500       20160708     0.355621
5       4005121       9983       20160706     0.437962
6       4347394       9983       20160716     0.437962
7       3094273      13602       20160727     0.396055
8       5139970       9983       20160729     0.121922
9       3237121      13602       20160703     0.396055
10      6224386       9983       20160716     0.317148
11      6488578      13602       20160712     0.437962
12      4164865       9983       20160703     0.355621
13      4164865       8059       20160706     0.396055
14      5468674       9983       20160713     0.121922
15      6258178       9144       20160706     0.437962
16      3659521       7341       20160727     0.216805
17      36

# 输出到制定的文件

In [48]:
dataTest.to_csv("sample_submission.csv", header=None, index=None)